![Quantinuum%20Logos_primary_blue_small.svg](attachment:Quantinuum%20Logos_primary_blue_small.svg)

# Mid-Circuit Measurement

This notebook contains an example using mid-circuit measurement using the Quantinuum machines.

## Repetition Code Circuit

The use of mid-circuit measurement is straightforward, note the use of `measure` and `reset` on the ancilla qubits. This example also utlizes conditional logic available with Quantinuum devices as well as Registers and IDs available in `pytket`. See [Classical and conditional operations](https://cqcl.github.io/pytket/manual/manual_circuit.html#classical-and-conditional-operations) and [Registers and IDs](https://cqcl.github.io/pytket/manual/manual_circuit.html#registers-and-ids) for additional examples.

In [ ]:
from pytket.circuit import Circuit, Qubit, Bit, OpType, reg_eq
from pytket.circuit.display import render_circuit_jupyter

Set up Repetition Code with mid-circuit measurement and corrections:<br>
`2    1    0 = data: data qubits`<br>
`*----*----*`<br>
`  ZZ   ZZ`<br>
`  1    0    = syndromes`<br>
`  0    0    = ancillas`

Set up circuit object

In [ ]:
circuit = Circuit(name="Repetition Code")

Add qubit register, the data qubits

In [ ]:
data = circuit.add_q_register("data", 3)

Add qubit register, the ancilla qubit

In [ ]:
ancilla = circuit.add_q_register("anc", 1)

Add classical registers for the syndromes

In [ ]:
syndrome = circuit.add_c_register("syndrome", 2)

Add classical registers for the output

In [ ]:
output = circuit.add_c_register("output", 3)

Prepare the logical state. Qubits always start in $| 0 \rangle$ and logical $ | 0 \rangle == | 000 \rangle$. So we already start in logical $| 0 \rangle$.<br><br>
Syndrome Extraction

In [ ]:
circuit.add_gate(OpType.Reset, ancilla)
circuit.CX(data[0], ancilla[0])
circuit.CX(data[1], ancilla[0])
circuit.Measure(ancilla[0], syndrome[0])

Syndrome Extraction

In [ ]:
circuit.add_gate(OpType.Reset, ancilla)
circuit.CX(data[1], ancilla[0])
circuit.CX(data[2], ancilla[0])
circuit.Measure(ancilla[0], syndrome[1])

Correction: if(syndromes==1) -> 01 -> check 0 bad -> X on qubit 0

In [ ]:
circuit.X(data[0], condition=reg_eq(syndrome, 1))

if(syndromes==2) -> 10 -> check 1 bad -> X on qubit 2

In [ ]:
circuit.X(data[2], condition=reg_eq(syndrome, 2))

if(syndromes==3) -> 11 -> check 1 and 2 bad -> X on qubit 1

In [ ]:
circuit.X(data[1], condition=reg_eq(syndrome, 3))

Measure out data qubits

In [ ]:
circuit.Measure(data[0], output[0])
circuit.Measure(data[1], output[1])
circuit.Measure(data[2], output[2])

In [ ]:
render_circuit_jupyter(circuit)

## Select Device

Login to the Quantinuum API using your credentials and check the device status.

In [ ]:
from pytket.extensions.quantinuum import QuantinuumBackend
machine = 'H1-2E'
backend = QuantinuumBackend(device_name=machine)
backend.login()

In [ ]:
print(machine, "status:", QuantinuumBackend.device_state(device_name=machine))

### Circuit Compilation

`pytket` includes many features for optimizing circuits. This includes reducing the number of gates where possible and resynthesizing circuits for a quantum computer's native gate set. See the `pytket` [User Manual](https://cqcl.github.io/pytket/manual/index.html) for more information on all the options that are available.<br><br>
Here the circuit is compiled with `get_compiled_circuit`, which includes optimizing the gates and resynthesizing the circuit to Quantinuum's native gate set. The `optimisation_level` sets the level of optimization to perform during compilation, check pytket documentation for more details.

In [ ]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=1)
render_circuit_jupyter(compiled_circuit)

## Submit and Run the Circuit

In [ ]:
n_shots = 100
handle = backend.process_circuit(compiled_circuit,
                                 n_shots=n_shots)
print(handle)

In [ ]:
status = backend.circuit_status(handle)
print(status)

In [ ]:
import json
result = backend.get_result(handle)
with open('pytket_mcmr_example.json', 'w') as file:
    json.dump(result.to_dict(), file)

## Analyze Results

We will now take the raw results and apply a majority vote to determine how many times we got 0 vs 1.<br>
First, define a majority vote function.

In [ ]:
def majority(result):
    """ Returns whether the output should be considered a 0 or 1. """
    if result.count(0) > result.count(1):
        return 0
    elif result.count(0) < result.count(1):
        return 1
    else:
        raise Exception('count(0) should not equal count(1)')

Now process the output

In [ ]:
result_output_cnts = result.get_counts([output[i] for i in range(output.size)])
result_output_cnts

In [ ]:
zeros = 0  # Counts the shots with majority zeros
ones = 0  # Counts the shots with majority ones
for out in result_output_cnts:
    m = majority(out)
    if m == 0:
        zeros += result_output_cnts[out]
    else:
        ones += result_output_cnts[out]

A logical zero was initialized, so our error rate should be number of ones / total number of shots: `ones/shots`

In [ ]:
p = ones / n_shots
print(f'The error-rate is: p = {p}')

<div align="center"> &copy; 2022 by Quantinuum. All Rights Reserved. </div>